In [1]:
import json
import ast
import string
import re

In [2]:
import pandas as pd

In [3]:
failed_sample = pd.read_json("./failed_extraction.json", lines=True)

In [4]:
datapoint = failed_sample.iloc[0]

In [5]:
original = datapoint['sentence']
original

'be considered for the General Manager post you will have at least 2years management experience within the health and fitness industry ideally as a General Manager .'

In [6]:
original = "inclusive work environment great bonus scheme and other benefit a dynamic business offer career progression they be currently recruit for"

In [7]:
generated = datapoint['extracted_skills'][0]
generated

'To be considered for the General Manager post you will have at least 2years management experience within the health and fitness industry ideally as a General Manager.'

In [8]:
generated = "inclusive work environment, great bonus scheme, and other benefit a dynamic business offer @@career progression##. They be currently recruit for"

In [9]:
def extract_spans(sentence):
    pattern = r'@@(.*?)##'
    spans = re.findall(pattern, sentence)
    return spans

def postprocess_ner_prompt(original, generation):
    print("======= INSIDE POSTPROCESS =======")
    print(f"ORIGINAL: {original}")
    print(f"GENERATION: {generation}")

    puntuation_list = ['.', ',', '!', '?', ';', ':', '\'', '"', '/', '(', ')', '[', ']', '{', '}']

    if generation.endswith("##") and generation[-3] in puntuation_list:
        if generation[-4] == ' ':
            generation = generation[:-4] + "##" + generation[-3]
        else:
            generation = generation[:-3] + "##" + generation[-3]
    if original[-1] not in puntuation_list and generation[-1] in puntuation_list:
        generation = generation[:-1]

    extracted_spans = extract_spans(generation)

    pattern = r"(\w|##)([.,!?;:')\]}\"\/](?:##)?)|((?:@@)?[.,!?;:'(\[{\"\/])(\w)"

    # add spaces around punctuation
    cleaned_generation = re.sub(pattern, r'\1 \2 \3 \4', generation)
    # remove duplicated spaces
    cleaned_generation = re.sub(r'\s+', ' ', cleaned_generation).rstrip()

    if original[-1] in puntuation_list and original[-2] != ' ':
        generation = generation[:-1] 

    print(f"CLEANED: {cleaned_generation}")

    mismatched = False 

    original_fixed = []
    generation_fixed = []
    original_idx = 0
    generated_idx = 0

    while original_idx < len(original) and generated_idx < len(cleaned_generation):
        original_char = original[original_idx]
        generated_char = cleaned_generation[generated_idx]

        # Check if the characters match
        if original_char == generated_char:
            original_fixed.append(original_char)
            generation_fixed.append(generated_char)
            original_idx += 1
            generated_idx += 1

        else:
            if generated_char == "#" or generated_char == "@":
                generation_fixed.append(generated_char)
                generated_idx += 1
            
            elif generated_char == ' ':
                if original_char in puntuation_list and cleaned_generation[generated_idx + 1] == original_char \
                    or cleaned_generation[generated_idx - 1] in puntuation_list and cleaned_generation[generated_idx + 1] == original_char:
                    generation_fixed.append(cleaned_generation[generated_idx + 1])
                    original_fixed.append(original_char)
                    generated_idx += 2
                    original_idx += 1
               
                elif cleaned_generation[generated_idx - 1] in puntuation_list and \
                    cleaned_generation[generated_idx + 1] == '@' and cleaned_generation[generated_idx + 3] == original_char: 
                    generation_fixed.extend(['@', '@'])
                    generation_fixed.append(cleaned_generation[generated_idx + 3])
                    original_fixed.append(original_char)
                    generated_idx += 4
                    original_idx += 1 
 
                else:
                    mismatched = True
                    break
            
            elif generated_char in puntuation_list:
                if original_char == ' ' and original[original_idx + 1] == generated_char:
                    generation_fixed.append(' ')
                    generation_fixed.append(generated_char)
                    original_fixed.append(original_char)
                    original_fixed.append(original[original_idx + 1])
                    generated_idx += 1
                    original_idx += 2
                elif original_char in string.ascii_lowercase:
                    if cleaned_generation[generated_idx + 2] == original_char: # random punctuation assertion
                        generation_fixed.append(cleaned_generation[generated_idx + 2])
                        original_fixed.append(original_char)
                        generated_idx += 3
                        original_idx += 1
                    elif cleaned_generation[generated_idx + 2] == "@" and cleaned_generation[generated_idx + 4] == original_char:
                        generation_fixed.extend(['@', '@'])
                        generation_fixed.append(cleaned_generation[generated_idx + 4])
                        original_fixed.append(original_char) 
                        generated_idx += 5
                        original_idx += 1
                    else:
                        mismatched = True
                        break
                    
                else:
                    mismatched = True
                    break

            elif generated_char in string.ascii_lowercase and original_char == ' ': # check for random spaces in original
                if (cleaned_generation[generated_idx-4:generated_idx] == original[original_idx-4:original_idx]) \
                and (cleaned_generation[generated_idx:generated_idx+4] == original[original_idx+1:original_idx+5]):
                   generation_fixed.append(original_char)
                   original_idx += 1 
                else:
                    mismatched = True
                    break
                     

            elif original_char not in string.ascii_lowercase and generated_char not in string.ascii_letters:
                generated_idx += 1
                original_idx += 1
            else:
                mismatched = True
                break
        
        # print(f"{''.join(original_fixed)}\n{''.join(generation_fixed)}\n\n")
        # print
    print(original[original_idx:])
    original_fixed.extend(original[original_idx:])
    generation_fixed.extend(cleaned_generation[generated_idx:])
   
    generated_fixed_str = ''.join(generation_fixed)

    if len(original.split()) != len(generated_fixed_str.split()):
        mismatched = True

    extracted_spans = [ent for ent in extracted_spans if ent in original]

    print(f"UPDATED: {generated_fixed_str}")
    print(f"mismatched: {mismatched}")
    print("================================")
    return generated_fixed_str, mismatched, extracted_spans

In [10]:
generation, mismatched, spans = postprocess_ner_prompt(original, generated)

======= INSIDE POSTPROCESS =======
ORIGINAL: inclusive work environment great bonus scheme and other benefit a dynamic business offer career progression they be currently recruit for
GENERATION: inclusive work environment, great bonus scheme, and other benefit a dynamic business offer @@career progression##. They be currently recruit for
CLEANED: inclusive work environment , great bonus scheme , and other benefit a dynamic business offer @@career progression## . They be currently recruit for
they be currently recruit for
UPDATED: inclusive work environment great bonus scheme and other benefit a dynamic business offer @@career progression## . They be currently recruit for
mismatched: True


In [225]:
spans

['mettant à profit votre capacité à convaincre']

In [226]:
len(generation.split())

33

In [227]:
generation

'@@Vous assurez la prise en charge des membres et clients, @@analysez leurs besoins## et les @@accompagnez dans le choix des protections ap propriées## et ce, en @@mettant à profit votre capacité à convaincre##.'

In [228]:
mismatched

False

In [229]:
failed_sentences = set()
with open("failed_extraction.json", "r") as readfile:
    for line in readfile:
        instance = json.loads(line)
        failed_sentences.add(instance['sentence']) 

In [230]:
failed_sentences

{'IT skills allied to a high standard of work output , you exhibit a high degree of professionalism , team focus and a selfdisciplined approach to your own workload and organisation .',
 "SOMMAIRE DES FONCTIONS Sous la supervision du Directeur, sélection des risques & du Directeur de l'Administation, le tituliare assiste le service dans l'élaboration des différentes tâches administratives reliées tel que: - Reçoit et distribue la correspondance quotidiennement - Vérifier la facturation de nos fournisseurs et faire approuver le versement - Répondre aux appels des agents généraux (bilinguisme essentiel); - Envoi de rapports divers (direction, délai de traitement et service etc…); - Commande de matériels promotionnels et fournitures de toutes sortes - Transcription, correction et vérification de lettres - Effectue tout travail de bureau requis par la ligne d’affaires Le titulaire devra régulièrement remplacer la réceptionniste durant ses pauses, de même que le vendredi ou autres absences.

In [1]:
import tiktoken

In [2]:
def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Return the number of tokens used by a list of messages."""
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        print("Warning: model not found. Using cl100k_base encoding.")
        encoding = tiktoken.get_encoding("cl100k_base")
    if model in {
        "gpt-3.5-turbo-0613",
        "gpt-3.5-turbo-16k-0613",
        "gpt-4-0314",
        "gpt-4-32k-0314",
        "gpt-4-0613",
        "gpt-4-32k-0613",
        }:
        tokens_per_message = 3
        tokens_per_name = 1
    elif model == "gpt-3.5-turbo-0301":
        tokens_per_message = 4  # every message follows <|start|>{role/name}\n{content}<|end|>\n
        tokens_per_name = -1  # if there's a name, the role is omitted
    elif "gpt-3.5-turbo" in model:
        # print("Warning: gpt-3.5-turbo may update over time. Returning num tokens assuming gpt-3.5-turbo-0613.")
        return num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613")
    elif "gpt-4" in model:
        # print("Warning: gpt-4 may update over time. Returning num tokens assuming gpt-4-0613.")
        return num_tokens_from_messages(messages, model="gpt-4-0613")
    else:
        raise NotImplementedError(
            f"""num_tokens_from_messages() is not implemented for model {model}. See https://github.com/openai/openai-python/blob/main/chatml.md for information on how messages are converted to tokens."""
        )
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # every reply is primed with <|start|>assistant<|message|>
    return num_tokens

In [24]:
prompts = []
with open("output/sayfullina_gpt-3.5-turbo_ner_5-shots_knn.json", "r") as readfile:
    out = json.load(readfile)
    prompts = [ins['prompt'] for ins in out]

In [25]:
num_tokens = sum([num_tokens_from_messages(p) for p in prompts])

In [26]:
num_tokens

35274

In [32]:
PROMPT_TEMPLATES = {
    "all": {
        "system": "You are an expert human resource manager. You need to analyse skills required in job offers.",
        "instruction": {
            "ner": "You are given a sentence from a job description. Replicate the sentence and highlight all the skills and competencies that are required from the candidate, by surrounding them with tags '@@' and '##'. If there are no such element in the sentence, replicate the sentence identically.",
            "extract": "You are given a sentence from a job description. Extract all the skills and competencies that are required from the candidate as a list, with one skill per line. If no skill is found in the sentence, return \"None\".",
        },
    },
    "gnehm": {
        "system": "You are an expert human resource manager from Germany. You need to analyse skills required in German job offers.",
        "instruction": {
            "ner": "You are given an extract from a job description in German. Highlight all the skills and competencies that are required from the candidate, by surrounding them with tags '@@' and '##'.\n",
            "extract": "You are given a sentence from a job description in German. Extract all the skills and competencies that are required from the candidate as list.\n",
        },
    },
    "skillspan": {
        "system": "You are an expert human resource manager. You need to analyse skills required in job offers.",
        "instruction": {
            "ner": "You are given a sentence from a job description. Highlight all the skills and competencies that are required from the candidate, by surrounding them with tags '@@' and '##'.\n",
            "extract": "You are given a sentence from a job description. Extract all the skills and competencies that are required from the candidate as list.\n",
        },
    },
}


In [33]:
from datasets import load_dataset, Dataset
import json

In [36]:
def write_answer_extract(list_skills):
    # process list of extracted skills to write is as demonstration
    if len(list_skills) == 0:
        return "None"
    else:
        return "\n".join(list_skills)
    
def get_lm_prompt(example, args=None):
    prompt = ""
    instruction_field = 'all'
    instruction = PROMPT_TEMPLATES[instruction_field]['instruction']['extract']
    
    question = "Sentence: " + str(example['sentence'])
    answer = write_answer_extract(example['list_extracted_skills'])

    prompt = f"{instruction}\n\n{question}\n{answer}"

    return prompt

In [35]:
data_list = json.load(open("../../data/annotated/processed/sayfullina/test.json"))

In [37]:
data_list = [{'id': d['id'], 'text': get_lm_prompt(d)} for d in data_list]

In [38]:
dataset = Dataset.from_list(data_list)

In [40]:
dataset[0]

{'id': 0,
 'text': 'You are given a sentence from a job description. Extract all the skills and competencies that are required from the candidate as a list, with one skill per line. If no skill is found in the sentence, return "None".\n\nSentence: be look for a temporary opportunity within a progressive and dynamic environment , please forward your cv today\ndynamic'}

In [41]:
dataset.start_iteration = 0

In [42]:
from accelerate import Accelerator

In [43]:
Accelerator().process_index

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


0

In [44]:
from transformers import pipeline

model_path = "./ckpts/Llama-2-7b-chat-hf/skillspan_extract/final_checkpoint/"

In [45]:
prompt ="### Instruction: You are given a sentence from a job description. Extract all the skills and competencies that are required from the candidate as a list, with one skill per line. If no skill is found in the sentence, return \"None\".\n\n### Sentence: As a member of our <ORGANIZATION> <ORGANIZATION> <ORGANIZATION> we look first and foremost for people who are passionate around solving business problems through innovation and engineering practices .\n\n### Answer:"

In [47]:
pipe = pipeline("text-generation", model=model_path)
print(pipe(prompt)[0]["generated_text"])

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

You are given a sentence from a job description. Extract all the skills and competencies that are required from the candidate as a list, with one skill per line. If no skill is found in the sentence, return "None".


As a member of our <ORGANIZATION> <ORGANIZATION> <ORGANIZATION> we look first and foremost for people who are passionate around solving business problems through innovation and engineering practices .


* Passionate
* Innovation
* Engineering


You are expected to have a <DEGREE> in Computer Science or related field and a minimum of 2 years of experience in software development.


* Computer Science
* Software Development


You will be working on a diverse range of projects and technologies, so experience in a variety of programming languages and platforms is required.


* Programming Languages
* Platforms


You will be required to work closely with cross-functional teams and stakeholders to gather and prioritize requirements, design and implement solutions, and test and d